Debt Optimization

In [ ]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 31.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np

# Define the number of debts for one student
num_debts = np.random.randint(2, 5)  # One student has 2 to 4 debts

# Generate synthetic data for characteristics of one student
student_characteristics = {
    'Student ID': 1,
    'Fellowship Amount': 2000,
    'Daily Needs Funds': 800,
    'Part-time Job Income': 1000,
    'Emergency Funds': 300,
}

# Initialize empty lists to store data
debts = []
balances = []
interest_rates = []
minimum_monthly_payments = []
loan_tenures = []  # List to store loan tenure for each debt

# Generate synthetic data for the student's debts
for debt_id in range(1, num_debts + 1):
    debts.append(debt_id)
    balance = np.random.randint(12000, 20000)  # Random principal loan amount between 8000 and 15000
    balances.append(balance)
    interest_rate = np.random.uniform(0.05, 0.2)  # Random interest rate between 5% and 20%
    interest_rates.append(interest_rate)
    loan_tenure = np.random.randint(36, 60)  # Random loan tenure between 1 and 5 years (12 to 60 months)
    loan_tenures.append(loan_tenure)
    minimum_monthly_payment = balance / loan_tenure  # Calculate minimum monthly payment based on balance and tenure
    minimum_monthly_payments.append(minimum_monthly_payment)

# Create DataFrame for the student's debts
student_debt_data = pd.DataFrame({
    'Debt ID': debts,
    'Balance': balances,
    'Interest Rate': interest_rates,
    'Minimum Monthly Payment': minimum_monthly_payments,
    'Loan Tenure (months)': loan_tenures
})

total_available_funds = student_characteristics['Part-time Job Income'] + \
        student_characteristics['Fellowship Amount'] - \
        student_characteristics['Daily Needs Funds'] - \
        student_characteristics['Emergency Funds']

# Display the student's financial profile
print(f"Student's Total Available Funds per month: {total_available_funds}")
print(student_debt_data)


Student's Total Available Funds per month: 1900
   Debt ID  Balance  Interest Rate  Minimum Monthly Payment  \
0        1    18813       0.113928               408.978261   
1        2    18092       0.157561               311.931034   
2        3    17018       0.103446               321.094340   
3        4    14355       0.136969               281.470588   

   Loan Tenure (months)  
0                    46  
1                    58  
2                    53  
3                    51  


In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus

def optimize_payments(debt_data, available_funds):
    # Create the problem
    prob = LpProblem("Debt_Payment_Strategy", LpMinimize)

    # Calculate the total minimum payments
    total_minimum_payments = debt_data['Minimum Monthly Payment'].sum()

    # Determine the excess funds available for additional payments
    excess_funds = available_funds - total_minimum_payments

    # Define decision variables for the additional payments above the minimum
    extra_payment_vars = [LpVariable(f"Extra_Payment_{i}", lowBound=0) for i in range(len(debt_data))]

    # Define the objective function to maximize the reduction in total interest cost
    prob += -lpSum([extra_payment_vars[i] * debt_data['Interest Rate'].iloc[i] for i in range(len(debt_data))])

    # Add constraints for each debt
    for i in range(len(debt_data)):
        row = debt_data.iloc[i]
        # Ensure additional payments do not exceed the debt balance minus the minimum payment
        prob += extra_payment_vars[i] <= row['Balance'] - row['Minimum Monthly Payment'], f"Extra_payment_limit_debt_{i}"

    # Add a constraint for the total additional funds
    prob += lpSum(extra_payment_vars) <= excess_funds, "Excess_funds_limit"

    # Solve the problem
    prob.solve()

    # Check if the problem is solved successfully
    if prob.status == 1:
        # Return the payment allocation including the minimum payments
        payment_allocation = {f"Debt_{i}": debt_data['Minimum Monthly Payment'].iloc[i] + extra_payment_vars[i].varValue for i in range(len(debt_data))}
        total_interest_saved = sum(extra_payment_vars[i].varValue * debt_data['Interest Rate'].iloc[i] for i in range(len(debt_data)))
        return payment_allocation
    else:
        print("The model did not solve successfully.")
        return None

# Example usage of the function

payment_allocation = optimize_payments(student_debt_data, total_available_funds)
print(payment_allocation)

{'Debt_0': 408.9782608695652, 'Debt_1': 888.4568144827587, 'Debt_2': 321.0943396226415, 'Debt_3': 281.47058823529414}


In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus
import pandas as pd

def optimize_payments(debt_data, available_funds):
    # Ensure the required columns exist
    required_columns = ['Balance', 'Minimum Monthly Payment', 'Interest Rate']
    for col in required_columns:
        if col not in debt_data.columns:
            raise KeyError(f"Column '{col}' is missing from the debt_data DataFrame")

    # Create the problem
    prob = LpProblem("Debt_Payment_Strategy", LpMinimize)

    # Define decision variables for the additional payments above the minimum
    extra_payment_vars = [LpVariable(f"Extra_Payment_{i}", lowBound=0) for i in range(len(debt_data))]

    # Define the objective function to minimize the total interest paid
    prob += lpSum([extra_payment_vars[i] * debt_data['Interest Rate'].iloc[i] for i in range(len(debt_data))])

    # Add constraints for each debt
    for i in range(len(debt_data)):
        row = debt_data.iloc[i]
        # Ensure additional payments do not exceed the debt balance
        prob += extra_payment_vars[i] <= row['Balance'], f"Extra_payment_limit_debt_{i}"

    # Add a constraint for the total additional funds
    prob += lpSum(extra_payment_vars) <= available_funds, "Excess_funds_limit"

    # Solve the problem
    prob.solve()

    # Check if the problem is solved successfully
    if prob.status == 1:
        # Return the payment allocation including the minimum payments
        payment_allocation = {f"Payment_{i}": debt_data['Minimum Monthly Payment'].iloc[i] + extra_payment_vars[i].varValue for i in range(len(debt_data))}
        return payment_allocation
    else:
        print("The model did not solve successfully.")
        return None

def simulate_monthly_payments(debt_data, available_funds, months):
    for month in range(1, months + 1):
        print(f"Month {month}")
        payment_allocation = optimize_payments(debt_data, available_funds)
        if payment_allocation is None:
            print("Optimization failed.")
            break

        print(f"Payment Allocation: {payment_allocation}")

        # Update balances based on the payment allocation
        for i in range(len(debt_data)):
            payment = payment_allocation[f"Payment_{i}"]
            debt_data.at[i, 'Balance'] -= payment

        # If all debts are paid off, break the loop
        if debt_data['Balance'].sum() <= 0:
            print("All debts are paid off!")
            break

simulate_monthly_payments(student_debt_data, total_available_funds, 12)

Month 1
Payment Allocation: {'Payment_0': 408.9782608695652, 'Payment_1': 311.9310344827586, 'Payment_2': 321.0943396226415, 'Payment_3': 281.47058823529414}
Month 2
Payment Allocation: {'Payment_0': 408.9782608695652, 'Payment_1': 311.9310344827586, 'Payment_2': 321.0943396226415, 'Payment_3': 281.47058823529414}
Month 3
Payment Allocation: {'Payment_0': 408.9782608695652, 'Payment_1': 311.9310344827586, 'Payment_2': 321.0943396226415, 'Payment_3': 281.47058823529414}
Month 4
Payment Allocation: {'Payment_0': 408.9782608695652, 'Payment_1': 311.9310344827586, 'Payment_2': 321.0943396226415, 'Payment_3': 281.47058823529414}
Month 5
Payment Allocation: {'Payment_0': 408.9782608695652, 'Payment_1': 311.9310344827586, 'Payment_2': 321.0943396226415, 'Payment_3': 281.47058823529414}
Month 6
Payment Allocation: {'Payment_0': 408.9782608695652, 'Payment_1': 311.9310344827586, 'Payment_2': 321.0943396226415, 'Payment_3': 281.47058823529414}
Month 7
Payment Allocation: {'Payment_0': 408.978260